##NER 모델링 및 학습

In [ ]:
!git clone https://github.com/ukairia777/tensorflow-bert-ner.git

Cloning into 'tensorflow-bert-ner'...
remote: Enumerating objects: 1531, done.
remote: Counting objects: 100% (1531/1531), done.
remote: Compressing objects: 100% (1528/1528), done.
remote: Total 1531 (delta 33), reused 1443 (delta 2), pack-reused 0
Receiving objects: 100% (1531/1531), 6.23 MiB | 12.89 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [ ]:
%ls

sample_data/  tensorflow-bert-ner/


In [ ]:
%cd tensorflow-bert-ner/

/content/tensorflow-bert-ner


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 37.0 MB/s eta 0:00:00


In [ ]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=e39197b3960755a71bbb2e46af8a2bc1865600a0b733d91e3f790b6fa87418cd
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [ ]:
import glob, re
import numpy as np
import codecs
import os
import json
from tqdm import tqdm
import tensorflow as tf
from seqeval.metrics import f1_score, classification_report
from transformers import shape_list, BertTokenizer, TFBertModel

In [ ]:
train_dataset = glob.glob('data/train_data/*.txt')
test_dataset = glob.glob('data/validation_data/*.txt')

In [ ]:
len(train_dataset)

1425

In [ ]:
len(test_dataset)

2

In [ ]:
def read_ner_data(dataset):

  tagged_sentences = []
  sentence = []

  for file_name in tqdm(dataset):
    f = open(file_name, 'r', encoding='utf-8-sig')
    for line in f:
      if len(line)==0 or line[:2]=="##" or line[0]=="\n":
          if len(sentence) > 0:
              tagged_sentences.append(sentence)
              sentence = []
          continue
      splits = line.split('\t') # 공백을 기준으로 속성을 구분한다.
      splits[-1] = re.sub(r'\n', '', splits[-1]) # 줄바꿈 표시 \n을 제거한다.
      word = splits[0]
      sentence.append([word, splits[-1]]) # 단어와 개체명 태깅만 기록한다.

  return tagged_sentences

In [ ]:
train_tagged_sentences = read_ner_data(train_dataset)
test_tagged_sentences = read_ner_data(test_dataset)

100%|██████████| 2/2 [00:00<00:00, 25.99it/s]


In [ ]:
print(len(train_tagged_sentences))
print(len(test_tagged_sentences))

23033
931


In [ ]:
train_tagged_sentences[0]

[['예', 'O'],
 ['를', 'O'],
 ['_', 'O'],
 ['들', 'O'],
 ['어', 'O'],
 ['_', 'O'],
 ['전기', 'O'],
 ['차', 'O'],
 ['의', 'O'],
 ['_', 'O'],
 ['생명', 'O'],
 ['인', 'O'],
 ['_', 'O'],
 ['배터리', 'O'],
 ['_', 'O'],
 ['부문', 'O'],
 ['의', 'O'],
 ['_', 'O'],
 ['세계적', 'O'],
 ['_', 'O'],
 ['기업', 'O'],
 ['_', 'O'],
 ['LG', 'B-ORG'],
 ['화학', 'I-ORG'],
 ['과', 'O'],
 ['_', 'O'],
 ['삼성', 'B-ORG'],
 ['SDI', 'I-ORG'],
 ['가', 'O'],
 ['_', 'O'],
 ['수혜', 'O'],
 ['를', 'O'],
 ['_', 'O'],
 ['보', 'O'],
 ['는', 'O'],
 ['_', 'O'],
 ['방식', 'O'],
 ['이', 'O'],
 ['다', 'O'],
 ['.', 'O']]

In [ ]:
def split_sentence_and_label(tagged_sentences):
  index = 0
  sentences = []
  ner_tags = []

  for tagged_sentence in tqdm(tagged_sentences):
    sentence = []
    ner_tag = []
    for word, label in tagged_sentence:
      if word == '_' or word == '\xad':
        continue
      sentence.append(word)
      ner_tag.append(label)

    assert len(sentence) == len(ner_tag), "Error with input length {} vs {}".format(len(sentence), len(ner_tag))

    sentences.append(sentence)
    ner_tags.append(ner_tag)
  
  return sentences, ner_tags

In [ ]:
train_sentences, train_labels = split_sentence_and_label(train_tagged_sentences)
test_sentences, test_labels = split_sentence_and_label(test_tagged_sentences)

100%|██████████| 931/931 [00:00<00:00, 44992.99it/s]


In [ ]:
print(train_sentences[:3])
print(train_labels[:3])

[['예', '를', '들', '어', '전기', '차', '의', '생명', '인', '배터리', '부문', '의', '세계적', '기업', 'LG', '화학', '과', '삼성', 'SDI', '가', '수혜', '를', '보', '는', '방식', '이', '다', '.'], ['[', '인천', '/', '경기', ']', '인천', '공항', '환승객', '대상', '‘', '맞춤', '형', '의료', '관광', '’', '뜬다'], ['경찰', '대학', '5', '기', '출신', '인', '조', '대장', '은', '지난', '12', '일', '대구', '수성', '경찰서', '에서', '근무하', '다가', '자원해', '울릉', '경비', '대장', '이', '됐', '다', '.']]
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-ORG', 'I-ORG', 'B-NOH', 'I-NOH', 'O', 'O', 'B-PER', 'O', 'O', 'B-DAT', 'I-DAT', 'I-DAT', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


In [ ]:
print(test_sentences[:3])
print(test_labels[:3])

[['SBS', '골프', '채널', '이준실', '본부', '장', '은', '4', '일', '“', '여자', '개막', '전', '인', '스포츠서울', '김영주', '골프', '여자', '오픈', '부터', '디지털', '고화질', '(', 'HD', ')', '방송', '을', '시작할', '예정', '이', '며', '중계', '홀', '을', '종전', '5', '~', '6', '개', '홀', '에서', '9', '개', '홀로', '늘릴', '것', '”', '이라고', '밝', '혔', '다', '.'], ['신지애', '-', '최경주', '부진한', '출발'], ['한국', '선수', '중', '에서', '는', '‘', '슈퍼', '땅콩', '’', '김미현', '(', 'KTF', ')', '이', '2', '언더파', '70', '타', '를', '기록해', '공동', '6', '위', '에', '오르', '는', '가장', '좋', '은', '출발', '을', '했', '다', '.']]
[['B-ORG', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'B-DAT', 'I-DAT', 'O', 'O', 'O', 'O', 'O', 'B-POH', 'I-POH', 'I-POH', 'I-POH', 'I-POH', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NOH', 'O', 'B-NOH', 'I-NOH', 'O', 'O', 'B-NOH', 'I-NOH', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PER', 'O', 'B-PER', 'O', 'O'], ['B-ORG', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'B-PER', 'O', 'B-ORG', 'O', 'O', 'B-NOH', 'I-NOH', 'B-NOH', 'I-NOH

In [ ]:
labels = set(tag for train_label in train_labels for tag in train_label)

In [ ]:
print(labels)

{'I-DAT', 'I-DUR', 'I-LOC', 'B-NOH', 'B-DUR', 'B-PNT', 'B-ORG', 'I-MNY', 'B-MNY', 'I-NOH', 'B-POH', 'B-PER', 'I-TIM', 'I-POH', 'I-ORG', 'B-TIM', 'I-PER', 'B-DAT', 'I-PNT', 'O', 'B-LOC'}


In [ ]:
tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}

In [ ]:
tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}

In [ ]:
print(tag_to_index)
print(index_to_tag)

{'I-DAT': 0, 'I-DUR': 1, 'I-LOC': 2, 'B-NOH': 3, 'B-DUR': 4, 'B-PNT': 5, 'B-ORG': 6, 'I-MNY': 7, 'B-MNY': 8, 'I-NOH': 9, 'B-POH': 10, 'B-PER': 11, 'I-TIM': 12, 'I-POH': 13, 'I-ORG': 14, 'B-TIM': 15, 'I-PER': 16, 'B-DAT': 17, 'I-PNT': 18, 'O': 19, 'B-LOC': 20}
{0: 'I-DAT', 1: 'I-DUR', 2: 'I-LOC', 3: 'B-NOH', 4: 'B-DUR', 5: 'B-PNT', 6: 'B-ORG', 7: 'I-MNY', 8: 'B-MNY', 9: 'I-NOH', 10: 'B-POH', 11: 'B-PER', 12: 'I-TIM', 13: 'I-POH', 14: 'I-ORG', 15: 'B-TIM', 16: 'I-PER', 17: 'B-DAT', 18: 'I-PNT', 19: 'O', 20: 'B-LOC'}


In [ ]:
tag_size = len(tag_to_index)
print('개체명 태깅 정보의 개수 :',tag_size)

개체명 태깅 정보의 개수 : 21


In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):

    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        tokens = []
        labels_ids = []
        for one_word, label_token in zip(example, label):
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        labels_ids += [pad_token_id_for_label]

        tokens = [cls_token] + tokens
        labels_ids = [pad_token_id_for_label] + labels_ids


        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label = labels_ids + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
X_train, y_train = convert_examples_to_features(train_sentences, train_labels, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 23033/23033 [00:52<00:00, 439.25it/s]


In [ ]:
X_test, y_test = convert_examples_to_features(test_sentences, test_labels, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 931/931 [00:00<00:00, 1134.12it/s]


In [ ]:
class TFBertForTokenClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFBertForTokenClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        all_output = outputs[0]
        prediction = self.classifier(all_output)

        return prediction

In [ ]:
def compute_loss(labels, logits):

  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
  active_loss = tf.reshape(labels, (-1,)) != -100
  reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
  labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)

  return loss_fn(labels, reduced_logits)

In [ ]:
model = TFBertForTokenClassification("klue/bert-base", num_labels=tag_size)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=compute_loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'bert.embeddings.position_ids', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
class F1score(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test

    def sequences_to_tags(self, label_ids, pred_ids):
      label_list = []
      pred_list = []

      for i in range(0, len(label_ids)):
        label_tag = []
        pred_tag = []

        for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
          if label_index != -100:
            label_tag.append(index_to_tag[label_index])
            pred_tag.append(index_to_tag[pred_index])
        
        label_list.append(label_tag)
        pred_list.append(pred_tag)

      return label_list, pred_list

    def on_epoch_end(self, epoch, logs={}):

      y_predicted = self.model.predict(self.X_test)
      y_predicted = np.argmax(y_predicted, axis = 2)

      label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)

      score = f1_score(label_list, pred_list)
      print(' - f1: {:04.2f}'.format(score * 100))
      print(classification_report(label_list, pred_list))

In [ ]:
f1_score_report = F1score(X_test, y_test)

In [ ]:
model.fit(
    X_train, y_train, epochs=3, batch_size=32,
    callbacks = [f1_score_report]
)

Epoch 1/3


30/30 [==============================] - 11s 274ms/step
 - f1: 86.66
              precision    recall  f1-score   support

         DAT       0.95      0.96      0.95       182
         DUR       0.65      0.64      0.65        50
         LOC       0.72      0.78      0.75       206
         MNY       0.80      1.00      0.89        20
         NOH       0.88      0.91      0.89      1007
         ORG       0.83      0.91      0.87       795
         PER       0.95      0.94      0.95       853
         PNT       0.81      0.78      0.80        60
         POH       0.53      0.63      0.57       214
         TIM       0.74      0.89      0.81        19

   micro avg       0.85      0.89      0.87      3406
   macro avg       0.79      0.84      0.81      3406
weighted avg       0.85      0.89      0.87      3406

720/720 [==============================] - 602s 808ms/step - loss: 0.1633
Epoch 2/3
30/30 [==============================] - 8s 275ms/step
 - f1: 86.13
              precis

##Mecab-ko

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/ner

/content/drive/MyDrive/ner


In [ ]:
# Only for korean dataset
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
/content/drive/MyDrive/ner/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 38.7 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-02-07 12:21:43--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c5:2ef4, 2406:da00:ff00::3403:4be7, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-k

In [ ]:
from konlpy.tag import Mecab

mecab = Mecab()

In [ ]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []

    for example in tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]

        tokens = [cls_token] + tokens
        label_mask = [pad_token_id_for_label] + label_mask


        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_masks = np.asarray(label_masks, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), label_masks

In [ ]:
def ner_prediction(examples, max_seq_len, tokenizer, lang='ko'):
  
  if lang == 'ko':
    examples = [mecab.morphs(sent) for sent in examples]
  else:
    examples = [sent.split() for sent in examples]

  X_pred, label_masks = convert_examples_to_features_for_prediction(examples, max_seq_len=128, tokenizer=tokenizer)
  y_predicted = model.predict(X_pred)
  y_predicted = np.argmax(y_predicted, axis = 2)

  pred_list = []
  result_list = []

  for i in range(0, len(label_masks)):
    pred_tag = []
    for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
      if label_index != -100:
        pred_tag.append(index_to_tag[pred_index])

    pred_list.append(pred_tag)

  for example, pred in zip(examples, pred_list):
    one_sample_result = []
    for one_word, label_token in zip(example, pred):
      one_sample_result.append((one_word, label_token))
    result_list.append(one_sample_result)

  return result_list

##맞춤법 검사

In [ ]:
import requests
import json
import time
import sys
from collections import OrderedDict
import xml.etree.ElementTree as ET

In [ ]:
from collections import namedtuple

# 조사와 어미도 단어로 처리함. 마땅한 영단어가 생각이 안 나서..
_checked = namedtuple('Checked',
    ['result', 'original', 'checked', 'errors', 'words', 'time'])


class Checked(_checked):
    def __new__(cls, result=False, original='', checked='', errors=0, words=[], time=0.0):
        return super(Checked, cls).__new__(
            cls, result, original, checked, errors, words, time)

    def as_dict(self):
        d = {
            'result': self.result,
            'original': self.original,
            'checked': self.checked,
            'errors': self.errors,
            'words': self.words,
            'time': self.time,
        }
        return d

    def only_checked(self):
        return self.checked

In [ ]:
base_url = 'https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn'


class CheckResult:
    PASSED = 0
    WRONG_SPELLING = 1
    WRONG_SPACING = 2
    AMBIGUOUS = 3
    STATISTICAL_CORRECTION = 4

In [ ]:

_agent = requests.Session()
PY3 = sys.version_info[0] == 3


def _remove_tags(text):
    text = u'<content>{}</content>'.format(text).replace('<br>','')
    if not PY3:
        text = text.encode('utf-8')

    result = ''.join(ET.fromstring(text).itertext())

    return result


def check(text):
    """
    매개변수로 입력받은 한글 문장의 맞춤법을 체크합니다.
    """
    if isinstance(text, list):
        result = []
        for item in text:
            checked = check(item)
            result.append(checked)
        return result

    # 최대 500자까지 가능.
    if len(text) > 500:
        return Checked(result=False)

    payload = {
        '_callback': 'window.__jindo2_callback._spellingCheck_0',
        'q': text
    }

    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
        'referer': 'https://search.naver.com/',
    }

    start_time = time.time()
    r = _agent.get(base_url, params=payload, headers=headers)
    passed_time = time.time() - start_time

    r = r.text[42:-2]

    data = json.loads(r)
    html = data['message']['result']['html']
    result = {
        'result': True,
        'original': text,
        'checked': _remove_tags(html),
        'errors': data['message']['result']['errata_count'],
        'time': passed_time,
        'words': OrderedDict(),
    }

    # 띄어쓰기로 구분하기 위해 태그는 일단 보기 쉽게 바꿔둠.
    # ElementTree의 iter()를 써서 더 좋게 할 수 있는 방법이 있지만
    # 이 짧은 코드에 굳이 그렇게 할 필요성이 없으므로 일단 문자열을 치환하는 방법으로 작성.
    html = html.replace('<span class=\'green_text\'>', '<green>') \
               .replace('<span class=\'red_text\'>', '<red>') \
               .replace('<span class=\'purple_text\'>', '<purple>') \
               .replace('<span class=\'blue_text\'>', '<blue>') \
               .replace('</span>', '<end>')
    items = html.split(' ')
    words = []
    tmp = ''
    for word in items:
        if tmp == '' and word[:1] == '<':
            pos = word.find('>') + 1
            tmp = word[:pos]
        elif tmp != '':
            word = u'{}{}'.format(tmp, word)
        
        if word[-5:] == '<end>':
            word = word.replace('<end>', '')
            tmp = ''

        words.append(word)

    for word in words:
        check_result = CheckResult.PASSED
        if word[:5] == '<red>':
            check_result = CheckResult.WRONG_SPELLING
            word = word.replace('<red>', '')
        elif word[:7] == '<green>':
            check_result = CheckResult.WRONG_SPACING
            word = word.replace('<green>', '')
        elif word[:8] == '<purple>':
            check_result = CheckResult.AMBIGUOUS
            word = word.replace('<purple>', '')
        elif word[:6] == '<blue>':
            check_result = CheckResult.STATISTICAL_CORRECTION
            word = word.replace('<blue>', '')
        result['words'][word] = check_result

    result = Checked(**result)

    return result

In [ ]:
def text_checker(error_text):
  if len(error_text) >= 500:
    ready_list = []
    check_list=[]
    while (len(error_text) >= 500):
      temp_str = error_text[:500]
      last_space = temp_str.rfind(' ')
      temp_str = error_text[0:last_space]
      ready_list.append(temp_str)

      error_text = error_text[last_space:]
    ready_list.append(error_text)

    for i in range(len(ready_list)):
      check_list.append(check(ready_list[i]).checked)
    
    sum_check_list = ' '.join(check_list)
    return sum_check_list

  else:
    return check(error_text).checked

In [ ]:
error_text = "상명대학교의 송재우는 두레미담에서 11시 30분에 점심을 먹었다."

In [ ]:
sent1=text_checker(error_text)

NER 개체명 인식

In [ ]:
# sent1 = '보이그룹들의 글로벌 행보가 뚜렷했던 과거와 달리 걸그룹들의 글로벌 인기도 늘어나면서 K팝의 인기가 다변화되고 있다.'
# sent2 = '최근 글로벌 K팝 시장에서는 큰 변화가 생기고 있다.'
# sent3 = '그룹 블랙핑크를 필두로 하는 걸그룹들의 활약이 두드러지고 있는 것'
# sent4 = '특히 신인 걸그룹 뉴진스, 르세라핌, 아이브를 비롯해 트와이스, 있지 등의 글로벌 활약이 뚜렷해지면서 보이그룹 중심이었던 글로벌 K팝 시장도 큰 변화를 보이고 있다.'
# sent5 = '가장 먼저 글로벌 음원 시장에서 큰 족적을 남긴 건 블랙핑크였다.'

In [ ]:
def ner_def(sent1):
  test_samples = [sent1]
  result_list = ner_prediction(test_samples, max_seq_len=128, tokenizer=tokenizer, lang='ko')
  return result_list

In [ ]:
result_list = ner_def(sent1)
result_list

100%|██████████| 1/1 [00:00<00:00, 753.29it/s]


1/1 [==============================] - 0s 171ms/step


[[('상명', 'B-ORG'),
  ('대학교', 'I-ORG'),
  ('의', 'O'),
  ('송재우', 'B-PER'),
  ('는', 'O'),
  ('두레', 'B-ORG'),
  ('미담', 'I-POH'),
  ('에서', 'O'),
  ('11', 'B-TIM'),
  ('시', 'I-TIM'),
  ('30', 'I-TIM'),
  ('분', 'I-TIM'),
  ('에', 'O'),
  ('점심', 'O'),
  ('을', 'O'),
  ('먹', 'O'),
  ('었', 'O'),
  ('다', 'O'),
  ('.', 'O')]]

##결과요약

In [ ]:
def keyword(ner_list):
    
    per_list = []
    org_list = []
    tim_list = []
    loc_list = []
    
    for i in range(len(ner_list[0])):
        word = ner_list[0][i][0]
        tag = ner_list[0][i][1]
        
        if len(tag) == 5:
            if tag[2:] == 'ORG':
                org_list.append(word)
                
        if len(tag) == 5:
            if tag[2:] == 'PER':
                per_list.append(word)
                
        if len(tag) == 5:
            if tag[2:] == 'TIM':
                tim_list.append(word)
                
        if len(tag) == 5:
            if tag[2:] == 'LOC':
                loc_list.append(word)
                
    org = ''
    for i in org_list:
        org += i

    per = ''
    for i in per_list:
        per += i

    tim = ''
    for i in tim_list:
        tim += i
        
    loc = ''
    for i in loc_list:
        loc += i
        
    return print(f"기관 : {org} \n장소 : {loc} \n시간 : {tim}\n이름 : {per}")


In [ ]:
# result_list2 = [[('상명', 'B-ORG'),
#   ('대학교', 'I-ORG'),
#   ('의', 'O'),
#   ('송재우', 'B-PER'),
#   ('는', 'O'),
#   ('두레미', 'O'),
#   ('담', 'I-LOC'),
#   ('에서', 'O'),
#   ('11', 'B-TIM'),
#   ('시', 'I-TIM'),
#   ('30', 'I-TIM'),
#   ('분', 'B-TIM'),
#   ('에', 'O'),
#   ('점심', 'O'),
#   ('을', 'O'),
#   ('먹', 'O'),
#   ('었', 'O'),
#   ('다', 'O'),
#   ('.', 'O')]]
  
keyword(result_list)    

기관 : 상명대학교두레 
장소 :  
시간 : 11시30분
이름 : 송재우


##음성인식

In [ ]:
!pip install speechrecognition 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 4.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
import speech_recognition as sr

sound_file = 'voice.wav'
def sst(sound_file):
  r = sr.Recognizer()
  with sr.AudioFile(sound_file) as source:
      audio = r.record(source)
  text_1 = r.recognize_google(audio, language='ko-KR')
  return text_1


sst(sound_file)

result2:
{   'alternative': [   {   'confidence': 0.87675762,
                           'transcript': '고려대학교 김광현은 11시 30분 점심을 먹었다'},
                       {'transcript': '고려대학교 김강현은 11시 30분 점심을 먹었다'},
                       {'transcript': '고려대학교 김광현은 11시 30분 먹었다'},
                       {'transcript': '고려대학교 김광현은 11시 30분'},
                       {'transcript': '고려대학교 김강현은 11시 30분 먹었다'}],
    'final': True}


'고려대학교 김광현은 11시 30분 점심을 먹었다'

##최종

인풋에 보이스를 넣으면 맞춤법 검사 후 ner 이후 요약까지

In [ ]:
#input
sound_file = 'voice9.wav'


def main(sound_file):
  voice2text = sst(sound_file)#음성을 텍스트로
  sent1=text_checker(voice2text)#맞춤법 수정
  result_list = ner_def(sent1)#한문장 넣으면 ner 결과
  keyword(result_list)#결과 프린트

main(sound_file)

result2:
{   'alternative': [   {   'confidence': 0.73669213,
                           'transcript': '고려대학교 김광현은 부산에서 11시 30분 회의를 진행했다'},
                       {'transcript': '고려대학교 김광현은 11시 30분 회의를 진행했다'},
                       {'transcript': '고려대학교 김강현은 부산에서 11시 30분 회의를 진행했다'},
                       {'transcript': '고려대학교 김광현은 11시 30분에'},
                       {'transcript': '고려대학교 김강현은 11시 30분 회의를 진행했다'}],
    'final': True}


100%|██████████| 1/1 [00:00<00:00, 366.73it/s]

1/1 [==============================] - 0s 59ms/step
기관 : 고려대학교 
장소 : 부산 
시간 : 11시30분
이름 : 김광현
